In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import tiktoken

In [2]:
!nvcc --version 

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Mon_Apr__3_17:16:06_PDT_2023
Cuda compilation tools, release 12.1, V12.1.105
Build cuda_12.1.r12.1/compiler.32688072_0


In [3]:
torch.__version__

'2.0.1+cu117'

In [4]:
torch.cuda.is_available()

True

In [ ]:
!nvidia-smi

In [6]:
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
block_size = 256
batch_size = 64
n_head = 6
n_layer = 6
n_embd = 384
dropout = 0.2

In [7]:
if torch.cuda.is_available():
    print(torch.cuda.get_device_properties(0).total_memory)
    torch.cuda.empty_cache()

15693381632


This follows a tutorial for "tiny shakespear" by Andrej Karpathy and is not my own work

In [8]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [9]:
print(len(text))

1115394


In [10]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [11]:
chars = sorted(set(text))
vocab_size = len(chars)
print(chars, vocab_size)

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'] 65


Making an "encoder" and decoder to turn chars to ints and then ints to chars

In [12]:
# s_to_i = { ch:i for i,ch in enumerate(chars)}
# i_to_s = { i:ch for i, ch in enumerate(chars)}

# encode = lambda s: [s_to_i[c] for c in s]
# decode = lambda l: ''.join([i_to_s[i] for i in l])

# secret = encode("hello my name is tinyshake!")
# print(secret)

In [13]:
enc = tiktoken.get_encoding("p50k_base")
vocab_size = enc.n_vocab

encode = lambda s: enc.encode(s)
decode = lambda l: enc.decode(l)

secret = encode("hello my name is tiny-shakespear!")
print(secret)

[31373, 616, 1438, 318, 7009, 12, 1477, 20621, 283, 0]


In [14]:
print(decode(secret))

hello my name is tiny-shakespear!


In [15]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([338022]) torch.int64
tensor([ 5962, 22307,    25,   198,  8421,   356,  5120,   597,  2252,    11,
         3285,   502,  2740,    13,   198,   198,  3237,    25,   198,  5248,
          461,    11,  2740,    13,   198,   198,  5962, 22307,    25,   198,
         1639,   389,   477, 12939,  2138,   284,  4656,   621,   284,  1145,
          680,    30,   198,   198,  3237,    25,   198,  4965,  5634,    13,
        12939,    13,   198,   198,  5962, 22307,    25,   198,  5962,    11,
          345,   760,   327,  1872,   385,  1526, 28599,   318,  4039,  4472,
          284,   262,   661,    13,   198,   198,  3237,    25,   198,  1135,
          760,   470,    11,   356,   760,   470,    13,   198,   198,  5962,
        22307,    25,   198,  5756,   514,  1494,   683,    11,   290,   356])


In [16]:
#train and validation split
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[:n]

In [17]:
train_data[:block_size +1]

tensor([ 5962, 22307,    25,   198,  8421,   356,  5120,   597,  2252,    11,
         3285,   502,  2740,    13,   198,   198,  3237,    25,   198,  5248,
          461,    11,  2740,    13,   198,   198,  5962, 22307,    25,   198,
         1639,   389,   477, 12939,  2138,   284,  4656,   621,   284,  1145,
          680,    30,   198,   198,  3237,    25,   198,  4965,  5634,    13,
        12939,    13,   198,   198,  5962, 22307,    25,   198,  5962,    11,
          345,   760,   327,  1872,   385,  1526, 28599,   318,  4039,  4472,
          284,   262,   661,    13,   198,   198,  3237,    25,   198,  1135,
          760,   470,    11,   356,   760,   470,    13,   198,   198,  5962,
        22307,    25,   198,  5756,   514,  1494,   683,    11,   290,   356,
         1183,   423, 11676,   379,   674,   898,  2756,    13,   198,  3792,
          470,   257, 15593,    30,   198,   198,  3237,    25,   198,  2949,
          517,  3375,   319,   470,    26,  1309,   340,   307, 

# Testing!

In [18]:
# x = train_data[:block_size]
# y = train_data[1:block_size+1]
# for t in range(block_size):
#     context = x[:t+1]
#     target = y[t]
    
#     print("when we input", context, "we want to get", target)

In [19]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i + 1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x, y

In [20]:
# for split in ['train', 'val']:
#     get_batch(split)

# Making the Model

In [21]:
if torch.cuda.is_available():
    torch.cuda.init()
    
class Head(nn.Module):

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        
        self.droput = nn.Dropout(dropout)

    def forward(self, x):

        B,T, C = x.shape
        #Single head self attention
        k = self.key(x)
        q = self.query(x)

        wei = q @ k.transpose(-2, -1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # this is a decoder, this means that if you remove this, it becomes an encoder, aka, it can talk to nodes in the future
        wei = F.softmax(wei, dim=-1)
        wei = self.droput(wei)
        v = self.value(x)
        out = wei @ v

        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(num_heads * head_size, n_embd)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out
    
    
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )
    
    def forward(self, x):
        return self.net(x)
    
    
class Block(nn.Module):
    
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.fwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)


    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.fwd(self.ln2(x))
        return x

class BigramLanguageModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.token_embeding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
    def forward(self, idx, targets=None):
        B, T = idx.shape
        token_emb = self.token_embeding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = token_emb + pos_emb
        x = self.blocks(x)
        logits = self.lm_head(x)
        
        if targets is None:
            loss = None
        else:
    
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            
        
        return logits, loss
    
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            
            logits, loss = self(idx_cond)
            
            logits = logits[:, -1, :]
            
            probs = F.softmax(logits, dim=1)
            
            idx_next = torch.multinomial(probs, num_samples=1)
            
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

# Load a previous model (Optional)

In [22]:
# model = torch.load("temp")
# model.eval()

In [23]:
# torch.distributed.init_process_group(backend='nccl', init_method='env://')

model = BigramLanguageModel()
model = model.to(device)

# model = DistributedDataParallel(model, device_ids=[0, 1])

m = nn.DataParallel(model)

In [24]:
@torch.no_grad()
def estimate_loss():
    out = {}
    m.eval()
    
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = m(X, Y) # <- Problem
            
            #average the loss
            if torch.cuda.device_count() > 1:
                temp_loss = torch.zeros(torch.cuda.device_count())
                for i in range(torch.cuda.device_count()):
                    temp_loss[i] = loss[i].item()
                    
                losses[k] = temp_loss.mean()
            else:
                losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

## Train the model!

In [25]:
optimizer = torch.optim.Adam(m.parameters(), lr=learning_rate)

In [ ]:
if torch.cuda.is_available():
    torch.cuda.init()
    
    
# sampler = DistributedSampler(dataset) if is_distributed else None
# loader = DataLoader(dataset, shuffle=(sampler is None),
#                     sampler=sampler)
# for epoch in range(start_epoch, n_epochs):
#     if is_distributed:
#         sampler.set_epoch(epoch)
#     train(loader)

    
for iter in range(max_iters):
    
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    
    xb, yb = get_batch('train')
    
    xb = xb.to(device)
    yb = yb.to(device)
    
    logits, loss = m(xb, yb)
    loss = loss.mean()
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

/home/benzshawelt/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


step 0: train loss 11.2992, val loss 11.2990


In [ ]:
#     xb, yb = get_batch('train')
    
#     logits, loss = m(xb, yb)
#     optimizer.zero_grad(set_to_none=True)
#     loss.backward()
#     optimizer.step()

In [ ]:
m = m.module
m.to('cuda:0')

context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

In [ ]:
torch.save(m, "temp")

In [ ]:
del m
del model

# Practice Below this point

In [ ]:
# torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B,T,C)
x.shape

In [ ]:
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b,t] = torch.mean(xprev, 0)

In [ ]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x

# Self-attention (making our heads)

In [ ]:
B,T, C = 4, 8, 32
x = torch.randn(B, T, C)

#Single head self attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)
q = query(x)
wei = q @ k.transpose(-2, -1)

tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float('-inf')) # this is a decoder, this means that if you remove this, it becomes an encoder, aka, it can talk to nodes in the future
wei = F.softmax(wei, dim=-1)

v = value(x)   
out = wei @ v

out.shape